<a href="https://colab.research.google.com/github/venkatanadikatla/pytorch/blob/main/RNN_with_SST_Better_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y torchtext
!pip install torchtext==0.6.0

Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached n

**Project Overview**

This project aims to develop a sentiment analysis model using a Bidirectionl Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) units. The model is trained on the Stanford Sentiment Treebank (SST) dataset, which consists of sentences labeled with sentiment categories (positive or negative or neutral). The objective is to classify the sentiment of sentences accurately.

Hyperparameters: **Please see the last cell of this ipynb file to see the hyperparameters change and the corresponding accuracy.**

Defining the RNN model: Experimented with various combinations of bidirectional LSTM, dropout, and optimizers like Adam, SGD, and RMSprop.

Firstly embedding that coverts words into dense vectors, BRNN lstm - this layer proccesses the embedded text sequences, fully connected layer - this to outputs the final predictions. Initiating a zerostate fuction to initialize the hidden state with zeros. Initiating the forward function to forward pass of the model.

Training the model: This training function iterates over the training data, performs forward and backward passes, updates the model parameters, and computes the training loss and accuracy.

Our goal was to improve the testing accuracy of the sentiment analysis model beyond the baseline model's performance. I've experimented with various hyperparameters to achieve the **test accuracy of 62.8%**





In [ ]:
import copy # this module provides functions to duplicate objects. It seems to be imported but not yet used in the code
import torch # The MAIN PyTorch package
from torch import nn # contains the essential modules for building NN in pytorch.
from torch import optim # Provides optimization algorithms, such as SGD, Adam, etc
import torchtext # A library for text processing that works well with pytorch (Currently a version 0.6.0 is being used in this code)
from torchtext import data # A module in torchtext used for data handling
from torchtext import datasets # provides datasets, including various NLP datasets.

TEXT = data.Field(sequential=True, batch_first=True, lower=True) # Sequential = True indicates that the data consists of sequences.
#Batch_first=True Ensure that batch dimenstion is the first dimension in the tensor. # lower=True Converts all the text to lowercase

LABEL = data.LabelField() # A subclass of Field specifically for handling labels in a classification task.

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL) #datasets.SST.splits - Loads the Standford Sentiment Treebank(SST) dataset and splits the dataset

# build dictionary
# build_vocab: Creates a mapping from tokens(words) to indices. This is essential for converting text data into numerical form that can be used by NN.
TEXT.build_vocab(train_data) # Builds the vocabulary for the text field using the training data.
LABEL.build_vocab(train_data)# Builds the vocabulary for the label field using the training data.

# hyperparameters
vocab_size = len(TEXT.vocab) # the size of the vocabulary (number of unique tokens in the training data)
label_size = len(LABEL.vocab) # the number of unique labels (classes) in the traning data
padding_idx = TEXT.vocab.stoi['<pad>'] # The index used for padding sequences to the same length
embedding_dim = 300  # The size of the word embeddings (dense vector representation of words)
hidden_dim = 256 # Size of the hidden layers in the model
dropout = 0.5
num_layers = 2
# patience =3

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=64)

# Data.bucketiterator.splits - Creates iterators for the training, validation and test sets.


downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 3.65MB/s]


extracting


In [ ]:
# # Print a few examples from the raw training data
# for i in range(50):
#     print(vars(train_data.examples[i]))
# import torch.nn.functional as F

In [ ]:
# # Adding attention mechanism here to help the model focus on different part of the input sequence when making predictions.
# class Attention(nn.Module):
#     def __init__(self, hidden_dim):
#         super(Attention, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
#         self.v = nn.Parameter(torch.rand(hidden_dim))

#     def forward(self, hidden, encoder_outputs):
#         timestep = encoder_outputs.size(1)
#         # h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
#         h = hidden.unsqueeze(1).repeat(1, timestep, 1)
#         print(f'h shape: {h.shape}')
#         attn_energies = self.score(h, encoder_outputs)
#         return F.functional.softmax(attn_energies, dim=1)

#     def score(self, hidden, encoder_outputs):
#         energy = torch.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
#         energy = energy.transpose(2, 1)
#         v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
#         energy = torch.bmm(v, energy)
#         return energy.squeeze(1)




In [ ]:
class RNN (torch.nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx,dropout, num_layers=2):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.label_size = label_size
    self.num_layers = num_layers
    self.dropout = nn.Dropout(dropout)
    self.padding_idx = padding_idx
    self.embedding = torch.nn.Embedding(vocab_size, embedding_dim,padding_idx = padding_idx)
    # self.rnn = torch.nn.RNN(embedding_dim,hidden_dim, nonlinearity='relu',batch_first=True)
    self.lstm = torch.nn.LSTM(embedding_dim,hidden_dim, num_layers=num_layers,dropout=dropout, batch_first=True, bidirectional=True)
    # self.attention = Attention(hidden_dim)
    self.fc = torch.nn.Linear(hidden_dim*2, output_dim) #hidden *2 because of BRNN


  def zero_state(self, batch_size):
    # Implement the function, which returns an initial hidden state.
    return (torch.zeros(self.num_layers*2, batch_size, self.hidden_dim),
            torch.zeros(self.num_layers*2, batch_size, self.hidden_dim))

  def forward(self, text):
    embedded = self.embedding(text)
    embedded = self.dropout(embedded)
    batch_size = text.size(0)
    h_0, c_0 = self.zero_state(batch_size)
    h_0, c_0 = h_0.to(text.device), c_0.to(text.device)  # Ensure the hidden state is on the same device as the input
    output, (hidden,cell) = self.lstm(embedded, (h_0, c_0))
    # Check dimensions
    # print(f'lstm_output shape: {output.shape}')  # [batch_size, seq_len, hidden_dim * 2]
    # print(f'hidden shape: {hidden.shape}')  # [num_layers * 2, batch_size, hidden_dim]


    # hidden = hidden[-1]
    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
    # print(f'concatenated hidden shape: {hidden.shape}')  # [batch_size, hidden_dim * 2]
    # attn_weights = self.attention(hidden[-1], output)
    # context = attn_weights.unsqueeze(1).bmm(output).squeeze(1)
    # Check dimensions
    # print(f'context shape: {context.shape}')  # [batch_size, hidden_dim * 2]

    return self.fc(self.dropout(hidden))





In [ ]:
def train_model(model,train_iter, optimizer, criterion, num_epochs =20):
  model.train()
  # best_val_loss = float('inf')
  # patience_counter = 0
  for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    for batch in train_iter:
      optimizer.zero_grad()
      text, labels = batch.text, batch.label
      output = model(text)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) ==labels).sum().item()

      _, predicted = torch.max(output.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    avg_loss = epoch_loss/len(train_iter)
    avg_acc = epoch_acc/len(train_iter.dataset)

    epoch_accuracy = 100*correct/total


    print(f' Epoch {epoch+1}, Train Loss: {avg_loss}, Train Accuracy: {epoch_accuracy}')

    #Early Stopping
    # val_loss = eval_model(model, val_iter, criterion)
    # if val_loss < best_val_loss:
    #   best_val_loss = val_loss
    #   patience_counter = 0

    # else:
    #   patience_counter +=1

    # if patience_counter >= patience:
    #   print('Early Stopping')
    #   break



In [ ]:
def eval_model(model, val_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in val_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(val_iter)
  avg_acc = epoch_acc/len(val_iter.dataset)

  epoch_accuracy = 100*avg_acc

  print(f'Validation Loss: {avg_loss},  Validation Accuracy: {epoch_accuracy}')
  return avg_loss




In [ ]:
def test_model(model, test_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in test_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(test_iter)
  avg_acc = epoch_acc/len(test_iter.dataset)

  epoch_accuracy = 100*correct/total

  print(f'Test Loss: {avg_loss},  Test Accuracy: {epoch_accuracy}')




In [ ]:
torch.manual_seed(42)
# model = RNN(vocab_size,embedding_dim,hidden_dim,label_size, padding_idx, )
model = RNN(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx,dropout, num_layers)
# optimizer = optim.Adam(model.parameters(), lr=0.0005)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# optimizer = optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# optimizer = optim.Adagrad(model.parameters(), lr=0.01)
# optimizer = optim.Adadelta(model.parameters(), lr=1.0, rho=0.9)
# optimizer = optim.Adamax(model.parameters(), lr=0.002)
# optimizer = optim.NAdam(model.parameters(), lr=0.002)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)


criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-3, step_size_up=2000, mode='triangular')

In [ ]:
train_model(model,train_iter, optimizer, criterion, num_epochs =20)

 Epoch 1, Train Loss: 1.0358599170820038, Train Accuracy: 46.125936329588015
 Epoch 2, Train Loss: 0.9945817313977142, Train Accuracy: 51.767322097378276
 Epoch 3, Train Loss: 0.9506405409592301, Train Accuracy: 56.08614232209738
 Epoch 4, Train Loss: 0.9000712179425937, Train Accuracy: 59.02387640449438
 Epoch 5, Train Loss: 0.8515530215270484, Train Accuracy: 61.96161048689139
 Epoch 6, Train Loss: 0.7979917757546724, Train Accuracy: 65.78885767790263
 Epoch 7, Train Loss: 0.7494230328211143, Train Accuracy: 67.39232209737828
 Epoch 8, Train Loss: 0.7062678027953675, Train Accuracy: 70.02574906367042
 Epoch 9, Train Loss: 0.6680171409649636, Train Accuracy: 71.80477528089888
 Epoch 10, Train Loss: 0.6300494319467402, Train Accuracy: 73.87640449438203
 Epoch 11, Train Loss: 0.5619215324743471, Train Accuracy: 76.79073033707866
 Epoch 12, Train Loss: 0.5329100060373989, Train Accuracy: 78.34737827715355
 Epoch 13, Train Loss: 0.4928752982349538, Train Accuracy: 80.10299625468164
 Epoch

In [ ]:
# Evaluate the model on validation data
eval_model(model, val_iter, criterion)



Validation Loss: 2.2240900529755487,  Validation Accuracy: 58.31062670299727


2.2240900529755487

In [ ]:
# Evaluate the model on test data
test_model (model, test_iter, criterion)

Test Loss: 1.9391816888536726,  Test Accuracy: 62.66968325791855


In [ ]:
test_data[43]

In [ ]:
#This function to show the test or anyunown data to check the quick check:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def get_specific_data_point(data, index):
    return data[index]

# Evaluate a specific data point
def evaluate_specific_data_point(model, data_point, text_field, label_field, criterion):
    model.eval()

    # Convert the data point to a tensor
    text = text_field.process([data_point.text]).to(device)
    label = torch.tensor([label_field.vocab.stoi[data_point.label]]).to(device)

    with torch.no_grad():
        output = model(text)
        loss = criterion(output, label)

        _, predicted = torch.max(output.data, 1)
        is_correct = (predicted.item() == label.item())

        print(f'Predicted: {label_field.vocab.itos[predicted.item()]}')
        print(f'Actual: {data_point.label}')
        print(f'Loss: {loss.item()}')
        print(f'Correct: {is_correct}')

# Get the specific data point
specific_data_point = get_specific_data_point(test_data, 2000)

# Evaluate the specific data point
evaluate_specific_data_point(model, specific_data_point, TEXT, LABEL, criterion)

Predicted: negative
Actual: negative
Loss: 5.722029527532868e-06
Correct: True


Used the following hyperparameters to increase the accuracy:

1. Changed the embedding_dim and hidden_dim to 256 (dense vector representation of words) check if this simple change in hyperparameters could change the accuracy. **However, test accuracy @ 56% was close to my baseline model accuracy @ 56.6%**
2. With the addition of item#1 added the dropout regularization technique @ 0.3 to see if this can help increasing the accuracy. **Improved the test accuracyt to 57%, however, this is slightly better than my baseline model accuracy.**
3. Now adding the LSTM to 2 layers, Increasing the learning rate to 0.0001 and removing the dropout all together to see if this will increase the acccuracy - **Performance is not better than item#2 infact the test accuracy dropped to 52%.**
4. Now adding the LSTM to 2 layers, Increasing the learning rate to 0.0001 and adding the dropout (0.3) to see if this will increase the acccuracy - **Apparently, there was no change in test accuracy. Reached test accuracy @ 51%.**
5. Updated the embedding dimension to 512 and hidden layers to 512, batch_size=128 and dropout =0.3 and lr=0.0001 - Still testing accuracy is not better than baseline model. **Performance is not better than baseline performance infact the test accuracy dropped to 53%**
6. Updating the batchsize =64 and increasing number of epochs =20 and leaving the same hyperparameters as item#5 other than batchsize and epochs for this attempt as wel - **didn't perform any better - test accuracy is 53%**
7. Used **BRNN LSTM and dropout rate and Adam optimizer weight decay** - didn't help the model test performance, reached only **51% accuracy.**
8. Used **BRNN LSTM and dropout ratge with SGD(optim.SGD(model.parameters(), lr=0.001, momentum=0.9)) - poor model test accuracy @ 42%**
9. Used BRNN LSTM and dropout rate 0.5 with optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9) - **poor model test accuracy @ 57% but slightly less than baseline model accuracy.**
10. Used BRNN LSTM and dropout rate 0.5 with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the better accuracy than the base line model - Test accuracy 59%***
11. Used BRNN LSTM, removed dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the better accuracy than the base line model - Test accuracy 59.5%***
12. Used BRNN LSTM, removed dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the poor accuracy than the base line model - Test accuracy 53.5%***
13. Used BRNN LSTM, added dropout with optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the better accuracy than base line model - test accuracy 58.9%***
14. Used BRNN LSTM, added dropout with optimizer = optim.Adagrad(model.parameters(), lr=0.01), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the  accuracy slightly better than base line model - test accuracy of 57%***
15. Used BRNN LSTM, added dropout with optimizer = optim.Adadelta(model.parameters(), lr=1.0, rho=0.9) , reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the  better accuracy than base line model - test accuracy of 58.2%***
16. Used BRNN LSTM, added dropout with optimizer = optim.Adamax(model.parameters(), lr=0.002) , reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the accuracy same as base line model - test accuracy of 56.7%***
17.  Used BRNN LSTM, added dropout with optimizer = optim.NAdam(model.parameters(), lr=0.002) , reduced the hidden_layers to 256 and reduced epochs to 10 - ***Got the accuracy slightly better than base line model - test accuracy of 57.8%***
18. Used BRNN LSTM, added dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4) and in addition added early stoping regularization technique , reduced the hidden_layers to 256 and  epochs to 20 - ***Got the accuracy slightly better than base line model - test accuracy of 57.2%***
19. Used BRNN LSTM, added dropout with optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9) and in addition removed early stoping regularization technique , reduced the hidden_layers to 256 and maintained epochs to 20 - ***Got the accuracy better than base line model - test accuracy of 62.6%***
20. Used BRNN LSTM, added dropout with optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9) and in addition removed early stoping regularization technique , increased word_embedding to 512, maintained the hidden_layers to 256 and maintained epochs to 20 - ***Got the accuracy better than base line model - test accuracy of 60.8%***



**BRNN with LSTM and Different Optimizers:** Experimented with various combinations of bidirectional LSTM, dropout, and optimizers like Adam, SGD, and RMSprop.

**Best Configuration:** **BRNN with LSTM, dropout rate 0.5, RMSprop optimizer (lr=0.001, alpha=0.9). Achieved the highest test accuracy of 62.6%.**


**Conclusion:**
Throughout the project, various hyperparameters and configurations were experimented with to enhance the model's test accuracy beyond the baseline. The use of bidirectional LSTM, dropout regularization, and RMSprop optimizer proved to be the most effective combination, achieving a test accuracy of 62.6%. This improvement demonstrates the potential of fine-tuning hyperparameters to enhance model performance, making sentiment analysis more accurate and reliable for business applications.